In [1]:
from pymongo import MongoClient
client = MongoClient()

In [2]:
client = MongoClient('localhost',
                    username='moira',
                    password='rootlogin')

In [3]:
client.list_database_names()

['admin', 'config', 'local', 'searchin']

In [4]:
db = client.searchEngine

In [5]:
db.list_collection_names()

[]

In [36]:
list(db['funnytest'].find())

[]

In [6]:
import numpy as np
import string

In [7]:
rpath = "./shakestxts/indextemps/"

In [8]:
metadata = np.load(rpath+'metadata.npy',allow_pickle='TRUE').item()
stem2doc_count = np.load(rpath+'stem2doc_count.npy',allow_pickle='TRUE').item()
stem2uid_count = np.load(rpath+'stem2uid_count.npy',allow_pickle='TRUE').item()
uid2detail = np.load(rpath+'uid2detail.npy',allow_pickle='TRUE').item()
uid2display = np.load(rpath+'uid2display.npy',allow_pickle='TRUE').item()
stem2idf = np.load(rpath+'stem2idf.npy',allow_pickle='TRUE').item()
uid2stemlist = np.load(rpath+'uid2stemlist.npy',allow_pickle='TRUE').item()

In [9]:
code2title = np.load(rpath+'code2title.npy',allow_pickle='TRUE').item()
code2synopsis = np.load(rpath+'code2synopsis.npy',allow_pickle='TRUE').item()

In [10]:
metadata

{'totalN': 73410, 'avgLen': 10.943045906552241}

In [81]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
replacement = ""
for i in range(32):
    replacement += " "
stopwfile = open("./stopw.txt")
stopw = []
for line in stopwfile.readlines():
    stopw.append(line.split("\n")[0])

In [82]:
def FreInDOfTerm(term, dID):
    if term in stem2uid_count.keys():
        if dID in stem2uid_count[term].keys():
            return stem2uid_count[term][dID]
    return 0

In [83]:
def processQuery(query):
    cleanedQ = []
    query = query.translate(str.maketrans(string.punctuation, replacement))
    for word in query.strip().split(" "):
        foldW = word.casefold()
        if foldW not in stopw:
            stemW = stemmer.stem(foldW)
            if stemW in stem2uid_count.keys():
                cleanedQ.append(stemW)
    return cleanedQ

In [84]:
def BM25(docID, query):
    bm25summ = 0
    for term in query:
        fieldLen = len(query)
        if term in stem2uid_count.keys():
            finDt = FreInDOfTerm(term, docID)
            bm25summ += ( stem2idf[term] * (finDt * 2.2) / ( finDt + 1.2 * (0.25 + 0.75 * ( fieldLen / metadata['avgLen'])) ) )
    return bm25summ

In [85]:
querytest = "love hate money power"
cleanQ = processQuery(querytest)

In [86]:
uid2bm25 = dict()
for uid in uid2stemlist.keys():
    uid2bm25[uid] = BM25(uid, cleanQ)

In [87]:
sortedResult = dict(sorted(uid2bm25.items(), key=lambda item: item[1], reverse=True))
outputSize = 5
for uid in sortedResult.keys():
    outputSize -= 1
    print(uid2display[uid])
    if outputSize < 0:
        break

 The people love me, and the sea is mine; My powers are crescent, and my auguring hope Says it will come to th’ full. Mark Antony In Egypt sits at dinner, and will make No wars without doors. Caesar gets money where He loses hearts. Lepidus flatters both,
 The strong necessity of time commands Our services awhile, but my full heart Remains in use with you. Our Italy Shines o’er with civil swords; Sextus Pompeius Makes his approaches to the port of Rome; Equality of two domestic powers Breed scrupulous faction; the hated grown to strength Are newly grown to love; the condemned Pompey,
 Follow thou the wars; defeat thy favor with an usurped beard. I say, put money in thy purse. It  continue her love to the Moor—put money in thy purse— nor he his to her. It was a violent commencement in her, and thou shalt see an answerable sequestration —put but money in thy purse. These Moors are changeable in their wills. Fill thy purse with money.
 Thou art sure of me. Go, make money. I have told thee

In [12]:
client.list_database_names()

['admin', 'config', 'local', 'searchin']

In [28]:
searchdb = client['searchin']
metac = searchdb['metadata']

In [29]:
metac.insert_one({'_id': 0, 'avgLen': metadata['avgLen']})

In [14]:
steminfo = searchdb['steminfo']

In [15]:
stemset = set()
for key in stem2doc_count.keys():
    stemset = stemset.union([key])

In [16]:
len(stemset)

14765

In [75]:
# mapping uid - index
uid2newindex = dict()
for index, uid in enumerate(uidlist):
    uid2newindex[uid] = index

In [76]:
def changeID(stemUIDC):
    newMap = dict()
    for uid in stemUIDC.keys():
        count = stemUIDC[uid]
        newMap[str(uid2newindex[uid])] = count
    sortednewMap = dict(sorted(newMap.items(), key=lambda item: item[1], reverse=True))
    topMap = dict()
    for item in sortednewMap.items():
        if item[1] <= 0:
            break
        topMap[item[0]] = item[1]
    return topMap

In [77]:
insertList = []
stemlist = list(stemset)
for index, stem in enumerate(stemlist):
    stemname = stem
    stemDocC = stem2doc_count[stem]
    stemIDF = stem2idf[stem]
    stemUIDC = stem2uid_count[stem]
    newinsert = dict()
    newinsert['_id'] = index
    newinsert['stemname'] = stemname
    newinsert['stemDocC'] = stemDocC
    newinsert['stemIDF'] = stemIDF
    newstemUIDC = changeID(stemUIDC)
    newinsert['stemUIDC'] = newstemUIDC
    insertList.append(newinsert)

In [78]:
insertList[0]

{'_id': 0,
 'stemname': 'unguid',
 'stemDocC': 2,
 'stemIDF': 4.467821131371679,
 'stemUIDC': {'72988': 1, '65735': 1}}

In [79]:
steminfo.insert_many(insertList)

In [19]:
cat2codef = open('./shakestxts/category2code.txt', 'r')
code2gen = dict()
for line in cat2codef.readlines():
    cat, codes = line.split(":")
    print(cat.strip())
    for code in codes.split(","):
        code2gen[code.strip()] = cat.strip()

Comedy
History
Tragedy
Poetry


In [126]:
# code2gen

In [20]:
workinfo = searchdb['workinfo']

In [21]:
insertList = []
for code in code2title.keys():
    ccode = code
    ctitle = code2title[code]
    cgenre = code2gen[code]
    csynop = ""
    if code in code2synopsis.keys():
        csynop = code2synopsis[code]
    cinfo = dict()
    cinfo['_id'] = ccode
    cinfo['title'] = ctitle
    cinfo['genre'] = cgenre
    cinfo['synopsis'] = csynop
    insertList.append(cinfo)

In [22]:
workinfo.insert_many(insertList)

In [24]:
uidset = set()
for key in uid2display.keys():
    uidset = uidset.union([key])

In [47]:
len(uidset)

73410

In [26]:
insertList = []
uidlist = list(uidset)
for index, uid in enumerate(uidlist):
    uuid = uid
    udisplay = uid2display[uid]
    udetailraw = uid2detail[uid]
    udetail = dict()
    if len(udetailraw) == 5:
        udetail['code'] = udetailraw[0]
        udetail['actnum'] = udetailraw[1]
        udetail['scnum'] = udetailraw[2]
        udetail['speaker'] = udetailraw[3]
        udetail['lineRange'] = udetailraw[4]
    elif len(udetailraw) == 2:
        udetail['code'] = udetailraw[0]
        udetail['linenum'] = udetailraw[1]
    else:
        print(udetailraw)
        print("wrong detail with id: "+uid)
    thisline = dict()
    thisline['_id'] = index
#     thisline['uid'] = uuid
    thisline['display'] = udisplay
    thisline['detail'] = udetail
    insertList.append(thisline)

In [27]:
lineinfo = searchdb['lineinfo']
lineinfo.insert_many(insertList)

In [30]:
searchdb.list_collection_names()

['steminfo', 'lineinfo', 'metadata', 'workinfo']

In [50]:
metac.find_one({'_id': 0})

{'_id': 0, 'avgLen': 10.943045906552241}

In [51]:
insertList = []
for index, stem in enumerate(stemlist):
    newinsert = dict()
    newinsert['_id'] = stem
    newinsert['stemID'] = index
    insertList.append(newinsert)

In [52]:
insertList

[{'_id': 'unguid', 'stemID': 0},
 {'_id': 'laert', 'stemID': 1},
 {'_id': 'forgotten', 'stemID': 2},
 {'_id': 'crime', 'stemID': 3},
 {'_id': 'stickest', 'stemID': 4},
 {'_id': 'gosl', 'stemID': 5},
 {'_id': 'crestfallen', 'stemID': 6},
 {'_id': 'vent', 'stemID': 7},
 {'_id': 'placeth', 'stemID': 8},
 {'_id': 'watchman', 'stemID': 9},
 {'_id': 'plowman', 'stemID': 10},
 {'_id': 'overslip', 'stemID': 11},
 {'_id': 'crusado', 'stemID': 12},
 {'_id': 'spinner', 'stemID': 13},
 {'_id': 'repeal', 'stemID': 14},
 {'_id': 'unthank', 'stemID': 15},
 {'_id': 'darksom', 'stemID': 16},
 {'_id': 'unbuckl', 'stemID': 17},
 {'_id': 'throca', 'stemID': 18},
 {'_id': 'alic', 'stemID': 19},
 {'_id': 'burial', 'stemID': 20},
 {'_id': 'figur', 'stemID': 21},
 {'_id': 'soundest', 'stemID': 22},
 {'_id': 'prophet', 'stemID': 23},
 {'_id': 'meat', 'stemID': 24},
 {'_id': 'woolli', 'stemID': 25},
 {'_id': 'madli', 'stemID': 26},
 {'_id': 'blush', 'stemID': 27},
 {'_id': 'prejudici', 'stemID': 28},
 {'_id': '

In [ ]:
stem2id

In [48]:
stem2idC = searchdb['stem2idC']

In [53]:
stem2idC.insert_many(insertList)

In [146]:
# preprocessor getting ready
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
replacement = ""
for i in range(32):
    replacement += " "
stopwfile = open("./stopw.txt")
stopw = []
for line in stopwfile.readlines():
    stopw.append(line.split("\n")[0])

In [147]:
def FreInDOfTerm(term, dID):
    if term in stem2uid_count.keys():
        if dID in stem2uid_count[term].keys():
            return stem2uid_count[term][dID]
    return 0

In [148]:
def processQuery(query):
    cleanedQ = []
    query = query.translate(str.maketrans(string.punctuation, replacement))
    for word in query.strip().split(" "):
        foldW = word.casefold()
        if foldW not in stopw:
            stemW = stemmer.stem(foldW)
            if stemW in stem2uid_count.keys():
                cleanedQ.append(stemW)
    return cleanedQ

In [149]:
def BM25(docID, query):
    bm25summ = 0
    for term in query:
        fieldLen = len(query)
        if term in stem2uid_count.keys():
            finDt = FreInDOfTerm(term, docID)
            bm25summ += ( stem2idf[term] * (finDt * 2.2) / ( finDt + 1.2 * (0.25 + 0.75 * ( fieldLen / metadata['avgLen'])) ) )
    return bm25summ

In [ ]:
querytest = "love hate money power"
cleanQ = processQuery(querytest)

In [ ]:
uid2bm25 = dict()
for uid in uid2stemlist.keys():
    uid2bm25[uid] = BM25(uid, cleanQ)

In [ ]:
sortedResult = dict(sorted(uid2bm25.items(), key=lambda item: item[1], reverse=True))
outputSize = 5
for uid in sortedResult.keys():
    outputSize -= 1
    print(uid2display[uid])
    if outputSize < 0:
        break

In [58]:
len(uidset)
uidlist = list(uidset)

In [59]:
metaC = searchdb['metadata']
metaC.insert_one({'uidset': uidlist})

# read from mongodb and query

In [13]:
searchdb = client['searchin']

In [14]:
item = searchdb.metadata.find_one({'avgLen': {'$exists': True}})
avgLen = item['avgLen']
avgLen

10.943045906552241

In [15]:
item = searchdb.metadata.find_one({'uidset': {'$exists': True}})
uidset = item['uidset']
len(uidset)

73410

In [16]:
searchdb.steminfo.find_one({'stemname': 'love'})

{'_id': ObjectId('605e3cefb968a86317d9ee02'),
 'stemname': 'love',
 'stemDocC': 2646,
 'stemIDF': 1.4430892417164294,
 'stemUIDC': {'31e50a50-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e50bcc-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e525ee-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e5370a-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e538c2-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e54880-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e55988-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e56b08-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e5a410-8e45-11eb-aa00-086d41cd4a14': 2,
  '31e5a604-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e5a668-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e5b63a-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e674b2-8e45-11eb-aa00-086d41cd4a14': 2,
  '31e674e4-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e688d0-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e68e70-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e6928a-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e692ee-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e6ca8e-8e45-11eb-aa00-086d41cd4a14': 1,
  '31e6d146-8e4

In [29]:
# preprocessor getting ready
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
replacement = ""
for i in range(32):
    replacement += " "
stopwfile = open("./stopw.txt")
stopw = []
for line in stopwfile.readlines():
    stopw.append(line.split("\n")[0])
    
def processQuery(query):
    cleanedQ = []
    relevantUid = set()
    query = query.translate(str.maketrans(string.punctuation, replacement))
    for word in query.strip().split(" "):
        foldW = word.casefold()
        if foldW not in stopw:
            stemW = stemmer.stem(foldW)
            print(stemW)
            cleanedQ.append(stemW)
    return cleanedQ

In [18]:
def BM25(docID, query, avgLen):
    bm25summ = 0
    for term in query:
        fieldLen = len(query)
        findterm = searchdb.steminfo.find_one({'stemname': term})
        finDt = 0
        if not findterm:
            continue
        if docID in findterm['stemUIDC'].keys():
            finDt = findterm['stemUIDC'][docID]
        bm25summ += ( findterm['stemIDF'] * (finDt * 2.2) / ( finDt + 1.2 * (0.25 + 0.75 * ( fieldLen / avgLen ))) )
    return bm25summ

In [24]:
def query_tree(query, limit):
    resultList = []
    outputSize = limit
    uid2bm25 = dict()
    avgLen = searchdb.metadata.find_one({'avgLen': {'$exists': True}})['avgLen']
    cleanQ = processQuery(query)
    print(cleanQ)
    for uid in searchdb.metadata.find_one({'uidset': {'$exists': True}})['uidset']:
        uid2bm25[uid] = BM25(uid, cleanQ, avgLen)
    sortedResult = dict(sorted(uid2bm25.items(), key=lambda item: item[1], reverse=True))
    for uid in sortedResult.keys():
        outputSize -= 1
        resultList.append(uid)
        if outputSize < 0:
            break
    return resultList

In [25]:
true_query = "love hate money power"

In [30]:
query_result_list = query_tree(true_query, 10)
query_result_list

love
hate
money
power
['love', 'hate', 'money', 'power']


Traceback (most recent call last):
  File "/Users/zq_mooreamor/anaconda3/envs/searchEngine/lib/python3.9/site-packages/pymongo/mongo_client.py", line 1752, in _process_periodic_tasks
    self._topology.update_pool(self.__all_credentials)
  File "/Users/zq_mooreamor/anaconda3/envs/searchEngine/lib/python3.9/site-packages/pymongo/topology.py", line 441, in update_pool
    server._pool.remove_stale_sockets(generation, all_credentials)
  File "/Users/zq_mooreamor/anaconda3/envs/searchEngine/lib/python3.9/site-packages/pymongo/pool.py", line 1152, in remove_stale_sockets
    sock_info = self.connect(all_credentials)
  File "/Users/zq_mooreamor/anaconda3/envs/searchEngine/lib/python3.9/site-packages/pymongo/pool.py", line 1193, in connect
    sock_info.ismaster(all_credentials)
  File "/Users/zq_mooreamor/anaconda3/envs/searchEngine/lib/python3.9/site-packages/pymongo/pool.py", line 546, in ismaster
    return self._ismaster(None, None, None, all_credentials)
  File "/Users/zq_mooreamor/anac

NotMasterError: Executor error during find command :: caused by :: interrupted at shutdown, full error: {'ok': 0.0, 'errmsg': 'Executor error during find command :: caused by :: interrupted at shutdown', 'code': 11600, 'codeName': 'InterruptedAtShutdown'}

In [22]:
def retrieveDetails(uid_list):
    full_info = dict()
    order = 1
    for uid in uid_list:
        lineinfo = searchdb['lineinfo'].find_one({"uid": uid})
        workinfo = searchdb['workinfo'].find_one({"code": lineinfo['detail']['code']})
        full_info_this = dict()
        full_info_this['order'] = order
        full_info_this['lineinfo'] = lineinfo
        full_info_this['workinfo'] = workinfo
        full_info[uid] = full_info_this
        order += 1
    return full_info
result_details = retrieveDetails(query_result_list)

In [23]:
result_details

{'322dea22-8e45-11eb-aa00-086d41cd4a14': {'order': 1,
  'lineinfo': {'_id': ObjectId('605e4fb6b968a86317d9f4de'),
   'uid': '322dea22-8e45-11eb-aa00-086d41cd4a14',
   'display': ' The brawns of Hercules; but his Jovial face— Murder in heaven! How? ’Tis gone. Pisanio,',
   'detail': {'code': 'Cym',
    'actnum': '4',
    'scnum': '2',
    'speaker': 'NOBODY',
    'lineRange': '384-385'}},
  'workinfo': {'_id': ObjectId('605e4d3fb968a86317d9f4da'),
   'code': 'Cym',
   'title': 'Cymbeline',
   'genre': 'Comedy',
   'synopsis': 'Cymbeline, which takes place in ancient Britain, is filled with hidden identities, extraordinary schemes, and violent acts. Long ago, the two sons of King Cymbeline were abducted, leaving Cymbeline with a daughter, Imogen. Cymbeline’s stepson, Cloten, is now his heir, and Cymbeline expects Imogen to marry him. She secretly marries Posthumus Leonatus instead.Banished from court, Posthumus makes a foolish bet on Imogen’s chastity, which leads to false evidence that 

# Pre-compute BM25

In [4]:
searchindb = client.searchin

In [5]:
searchindb.list_collection_names()

['stem2idC', 'lineinfo', 'metadata', 'workinfo', 'steminfo']

In [ ]:
# no